# 💡 Hybrid Report Generator: Natural Language + Structured Metadata
This notebook uses LLM (OpenAI-compatible) to generate Spark SQL views from both:
- Natural Language (e.g. _"Show average amount by account type"_)
- Structured metadata (dimensions, measures, filters)

Each view is auto-tagged and ready to use in dashboards.

In [0]:
import json
from datetime import datetime
from openai import OpenAI

# Set up OpenAI client
OPENAI_API_KEY = dbutils.secrets.get(scope="llm", key="openai_api_key")
client = OpenAI(api_key=OPENAI_API_KEY)

In [0]:
# Path to metadata file (DBFS or ADLS)
metadata_path = "abfss://kyc-data@reprotingfactorydl.dfs.core.windows.net/finance/kyc/metadata/report_definitions_llm.json"
# Load metadata
try:
    metadata_df = spark.read.option("multiline", "true").json(metadata_path)
    report_defs = metadata_df.toJSON().map(json.loads).collect()
except Exception as e:
    raise Exception(f"Failed to load metadata file: {metadata_path}. Error: {e}")

In [0]:
# Hybrid prompt builder
def build_prompt(report, schema):
    schema_str = ", ".join([f"{col['name']} ({col['type']})" for col in schema])
    if "natural_language" in report:
        return f"""
        You are a Spark SQL generator.
        Query: {report['natural_language']}
        Table: finance.kyc_ml.customer_enriched
        Schema: {schema_str}
        Return only valid SQL without explanation or markdown.
        """

    dims = ', '.join(report.get("dimensions", []))
    meas = ', '.join(report.get("measures", []))
    filts = report.get("filters", "")
    return f"""
        Generate a SQL query on finance.kyc_ml.customer_enriched with the following schema:{schema_str}
        Dimensions: {dims}
        Measures: {meas}
        Filters: {filts}
        Return only valid SQL without explanation or markdown.
    """

In [0]:
# Extract schema from source table
source_df = spark.table("finance.kyc_ml.customer_enriched")
table_schema = source_df.schema.jsonValue()['fields']

# Main loop to generate views from LLM-generated SQL
for report in report_defs:
    prompt = build_prompt(report, table_schema)

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    generated_sql = response.choices[0].message.content.strip()
    view_name = f"finance.kyc_ml.{report['report_name']}"
    print(f"Creating view: {view_name}")
    #print(f"CREATE OR REPLACE VIEW {view_name} AS {generated_sql}")

    spark.sql(f"CREATE OR REPLACE VIEW {view_name} AS {generated_sql}")

    spark.sql(f"""
        ALTER VIEW {view_name}
        SET TBLPROPERTIES (
            'lineage.generated_by' = 'LLM',
            'lineage.description' = '{report.get("natural_language", report.get("description", ""))}',
            'lineage.created_at' = '{datetime.now().isoformat()}'
        )
    """)